# En Yakın Komşu ALgoritması

* Tahminler gözlem benzerliğine göre yapılır
* çevrenizdeki beş kişinin ortalamasısınızdır

# KNN Basamakları
* Komşu sayısını belirle (k)
* Bilinmeyen nokta ile diğer tüm noktalar arası uzaklıoğı hesapla
* uzaklıkları sırala ve belirlene k sayısına göre en yakın olan k gözlemi seç
* Sınıflandırma ise en sıksınıf, regresyon ise ortalam değeri tahmin değeri olarak ver

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [11]:
hit=pd.read_csv("Hitters.csv")
df=hit.copy()
df=df.dropna()
dms=pd.get_dummies(df[["League","Division","NewLeague"]])
y=df["Salary"]
X_=df.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X=pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25,random_state=42)


In [12]:
knn=KNeighborsRegressor().fit(X_train,y_train)

In [13]:
# farklı komşuluk sayısına göre deiğik sonuçalr alırız. optimize edilmesi gereken paremetre komşuluk sayısıdır

In [15]:
y_pred=knn.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test))

426.6570764525201

In [16]:
# hocam farklı k değerlerinde farklı sonuçlar alırız dedik bakalım nasıl oluyormuş bu durum
#brut force yöntemi ile deneyelim

RMSE=[]

for k in range(10):
    k=k+1
    RMSE.append(np.sqrt(mean_squared_error((KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)).predict(X_train),y_train)))
    print("k = ",k,"için RMSE değeri = ",RMSE[k-1])

k =  1 için RMSE değeri =  0.0
k =  2 için RMSE değeri =  179.52761335480352
k =  3 için RMSE değeri =  205.20157172291863
k =  4 için RMSE değeri =  220.5139794876305
k =  5 için RMSE değeri =  239.6467132541376
k =  6 için RMSE değeri =  243.5904190007242
k =  7 için RMSE değeri =  258.1478781634636
k =  8 için RMSE değeri =  266.05374203349805
k =  9 için RMSE değeri =  269.73782093553376
k =  10 için RMSE değeri =  271.2798300436963


# Model Tuning

In [17]:
knn_params={"n_neighbors":np.arange(1,30,1)}
KNN=KNeighborsRegressor()
knn_model=GridSearchCV(KNN,knn_params,cv=10)
knn_model.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [22]:
knn_model.best_params_

{'n_neighbors': 8}

In [21]:
RMSE =[]
RMSE_CV=[]
for k in range(10):
    k=k+1
    knn=KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)
    rmse=np.sqrt(mean_squared_error(y_train,knn.predict(X_train)))
    rmse_cv=np.sqrt(-1*cross_val_score(knn,X_train,y_train,cv=10,scoring="neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    print("k için rmse = ",RMSE[k-1],"rmse_cv = ",RMSE_CV[k-1])

k için rmse =  0.0 rmse_cv =  325.3947514706382
k için rmse =  179.52761335480352 rmse_cv =  293.24000183333817
k için rmse =  205.20157172291863 rmse_cv =  283.7486667487823
k için rmse =  220.5139794876305 rmse_cv =  286.3240222024089
k için rmse =  239.6467132541376 rmse_cv =  290.0705466132226
k için rmse =  243.5904190007242 rmse_cv =  298.1263115575851
k için rmse =  258.1478781634636 rmse_cv =  294.77070479194987
k için rmse =  266.05374203349805 rmse_cv =  291.98672028891235
k için rmse =  269.73782093553376 rmse_cv =  295.7162739573105
k için rmse =  271.2798300436963 rmse_cv =  301.31047022701154


In [24]:
#hocam biz GridSearchCV metodunu kullandığımızda ızgara tarama yöntemiyle en iyi k değerini buluruz
#şimdi yukarıda elle yaptığımızda en iyi k değerini 2 bulmuştuk GridSearchCV göre ise 8 bulduk hocam işte 
#bu yüzden mutlaka valide edilmiş scora bakmamız gerekiyor
#hemen yukarıda yaptığımız iteration'da da normal ve valşde arasındaki farkı rahatlıkla gözlemleyebiliriz
#şöyle de bir durum normalde sonuç 179 ile 271 arasınd geziyor -> 92 lik fark var
#valide score'da ise 283 ile 301 arası ->18 lik fark var
#bu farkın az olmasıda daah güvenilir olduğunun bir kanıtı

In [25]:
#bu yukarıdakş döngü tamamen gözlem öğrenme amaçlı
#GridSearchCV ile best k değerini bulduk şimdi final modelimizi kuralım

In [27]:
knn_params={"n_neighbors":np.arange(1,30,1)}
KNN=KNeighborsRegressor()
knn_cv=GridSearchCV(KNN,knn_params,cv=10).fit(X_train,y_train)
knn_final=KNeighborsRegressor(n_neighbors=knn_cv.best_params_["n_neighbors"]).fit(X_train,y_train)

In [28]:
np.sqrt(mean_squared_error(y_test,knn_final.predict(X_test)))

413.7094731463598